In [69]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 

from sklearn import datasets
from sklearn.model_selection import train_test_split

## Loading and merging data

In [70]:
# read in csv
sygefrav = pd.read_csv('/Users/laura/Desktop/data_dat_sci_exam/Sygefravær.csv', sep=';')
# drop the column LoenafdelingNiveau1Tekst	
sygefrav = sygefrav.drop(columns=['LoenafdelingNiveau1Tekst']) # just displaying that it is Gødstrup
# convert dato to timestamps with pd
sygefrav['Timestamp'] = pd.to_datetime(sygefrav['Dato'])

# make column called Date whih is the latter to values in the column Dato splitted by a -
sygefrav['Year'] = sygefrav['Dato'].apply(lambda x: x.split('-')[0])
sygefrav['Month'] = sygefrav['Dato'].apply(lambda x: x.split('-')[1])
sygefrav['Day'] = sygefrav['Dato'].apply(lambda x: x.split('-')[2])

# from now on we use timestamp
sygefrav = sygefrav.drop(columns=['Dato']) 

# order the data by timestamp
sygefrav = sygefrav.sort_values(by=['Timestamp'])
sygefrav.head(6)

,LoenafdelingNiveau2Tekst,Aldersgruppe,Stilling_niv1,Fravaeraarsag_gruppe,Koen,Antal_Timer,Antal_Personer,Timestamp,Year,Month,Day
24290,Øjensygdomme (SR_ØJEN),40-50,Plejepersonale (8M_02),Sygefravær,Kvinde,0.0,1,2021-01-01,2021,01,01
69991,Ortopædkirurgi (SR_ORTO),30-40,Plejepersonale (8M_02),Sygefravær,Kvinde,16.5,3,2021-01-01,2021,01,01
69988,Hjertesygdomme (SR_HJSY),50-60,Plejepersonale (8M_02),Sygefravær,Kvinde,0.0,1,2021-01-01,2021,01,01
69987,"Øre-, Næse- og Halskirurgi (SR_ØNH)",60-70,Plejepersonale (8M_02),Sygefravær,Kvinde,0.0,1,2021-01-01,2021,01,01
92038,Operation og Intensiv (SR_ANÆS),30-40,Lægepersonale (8M_01),Sygefravær,Kvinde,0.0,1,2021-01-01,2021,01,01
33821,Mave- og Tarmkirurgi (SR_KIRU),60-70,Plejepersonale (8M_02),Sygefravær,Kvinde,0.0,1,2021-01-01,2021,01,01


### Incorporate public holidays and weekends in the data

In [71]:
# find weekends)
sygefrav['is_weekend'] = 0

# Defining the initial weekends from the given start date
start_weekends = pd.to_datetime(['2021-01-02', '2021-01-03'])

# Generate all possible weekend dates from the initial weekends up to the latest date in your data
all_weekends = pd.date_range(start=min(start_weekends), end=sygefrav['Timestamp'].max(), freq='W-SAT').union(
                pd.date_range(start=min(start_weekends), end=sygefrav['Timestamp'].max(), freq='W-SUN'))

# Set the 'is_weekend' flag to 1 for all rows where the timestamp is a weekend
sygefrav.loc[sygefrav['Timestamp'].isin(all_weekends), 'is_weekend'] = 1


In [72]:
# checking the weekends
sygefrav[sygefrav['Timestamp'] == '2023-05-21'].head(3)

,LoenafdelingNiveau2Tekst,Aldersgruppe,Stilling_niv1,Fravaeraarsag_gruppe,Koen,Antal_Timer,Antal_Personer,Timestamp,Year,Month,Day,is_weekend
59488,Urinvejskirurgi (SR_URLO),60-70,Plejepersonale (8M_02),Sygefravær,Kvinde,0.00,1,2023-05-21,2023,05,21,1
34727,Operation og Intensiv (SR_ANÆS),50-60,Plejepersonale (8M_02),Sygefravær,Kvinde,30.56,10,2023-05-21,2023,05,21,1
137029,Operation og Intensiv (SR_ANÆS),40-50,Plejepersonale (8M_02),Sygefravær,Kvinde,8.00,12,2023-05-21,2023,05,21,1


In [73]:
# use timestamp to make a weekday index
sygefrav['weekday'] = sygefrav['Timestamp'].dt.dayofweek
# make a column with the name of the weekday
sygefrav['weekday_name'] = sygefrav['Timestamp'].dt.day_name()

now for **holidays** - these are based of on the closing days of banks: https://www.nationalbanken.dk/da/vores-arbejde/stabile-priser-pengepolitik-og-dansk-oekonomi/banklukkedage

In [74]:
holidays = [
    '2024-01-01',  # New Year's Day
    '2024-03-28',  # Skærtorsdag
    '2024-03-29',  # Langfredag
    '2024-04-01',  # påskedag
    '2024-04-02',  # påskedag
    '2024-05-09',  # Kristi himmelfartsdag
    '2024-05-20',  # Pinsedag
    '2024-12-24',  # Juleaftensdag
    '2024-12-25',  # Juledag
    '2024-12-26',  # 2 juledag
    '2024-12-31'   # Bank holiday

    ##### 2023 #####
    '2023-01-01',  # New Year's Day
    '2023-04-06',  # Skærtorsdag
    '2023-04-07',  # Langfredag
    '2023-04-09',  # påskedag
    '2023-04-10',  # påskedag
    '2023-05-18',  # Kristi himmelfartsdag
    '2023-05-28',  # Pinsedag
    '2023-12-24',  # Juleaftensdag
    '2023-12-25',  # Juledag
    '2023-12-26',  # 2 juledag
    '2023-12-31'   # Bank holiday

    ##### 2022 #####
    '2022-01-01',  # New Year's Day
    '2022-04-14',  # Skærtorsdag
    '2022-04-15',  # Langfredag
    '2022-04-17',  # påskedag
    '2022-04-18',  # påskedag
    '2022-05-26',  # Kristi himmelfartsdag
    '2022-06-05',  # Pinsedag
    '2022-12-24',  # Juleaftensdag
    '2022-12-25',  # Juledag
    '2022-12-26',  # 2 juledag
    '2022-12-31'   # Bank holiday

    ##### 2021 #####
    '2021-01-01',  # New Year's Day
    '2021-04-04',  # Skærtorsdag
    '2021-04-05',  # Langfredag
    '2021-04-07',  # påskedag
    '2021-04-08',  # påskedag
    '2021-05-13',  # Kristi himmelfartsdag
    '2021-05-23',  # Pinsedag
    '2021-12-24',  # Juleaftensdag
    '2021-12-25',  # Juledag
    '2021-12-26',  # 2 juledag
    '2021-12-31'   # Bank holiday
]



# Set the 'is_holiday' flag to 1 for all rows where the timestamp is a holiday
sygefrav['is_holiday'] = 0
sygefrav.loc[sygefrav['Timestamp'].dt.strftime('%Y-%m-%d').isin(holidays), 'is_holiday'] = 1

# checking the holidays
sygefrav[sygefrav['Timestamp'] == '2021-05-23'].head(3)

,LoenafdelingNiveau2Tekst,Aldersgruppe,Stilling_niv1,Fravaeraarsag_gruppe,Koen,Antal_Timer,Antal_Personer,Timestamp,Year,Month,Day,is_weekend,weekday,weekday_name,is_holiday
101376,Øjensygdomme (SR_ØJEN),40-50,Lægepersonale (8M_01),Sygefravær,Mand,0.0,2,2021-05-23,2021,05,23,1,6,Sunday,1
5396,Medicinsk Afdeling (SR_MEDC),20-30,Plejepersonale (8M_02),Sygefravær,Kvinde,0.0,4,2021-05-23,2021,05,23,1,6,Sunday,1
13437,Praksislæger (SR_REFU),30-40,Lægepersonale (8M_01),Sygefravær,Kvinde,0.0,2,2021-05-23,2021,05,23,1,6,Sunday,1


### Additional School Holidays 
week 42, week 7, and week 28, 29, 30 are typically the school holidays in denmark

In [75]:
school_holiday_weeks = [7, 42, 28, 29, 30] 
sygefrav['is_school_holiday'] = 0

# Loop through the school holiday weeks and update the 'is_school_holiday' column
for week in school_holiday_weeks:
    sygefrav.loc[sygefrav['Timestamp'].dt.isocalendar().week == week, 'is_school_holiday'] = 1

# checking the school holidays
sygefrav[sygefrav['Timestamp'] == '2024-02-15'].head(3) # the 15th of February 2024 is in week 7

,LoenafdelingNiveau2Tekst,Aldersgruppe,Stilling_niv1,Fravaeraarsag_gruppe,Koen,Antal_Timer,Antal_Personer,Timestamp,Year,Month,Day,is_weekend,weekday,weekday_name,is_holiday,is_school_holiday
127610,Hjertesygdomme (SR_HJSY),60-70,Plejepersonale (8M_02),Sygefravær,Kvinde,7.75,1,2024-02-15,2024,02,15,0,3,Thursday,0,1
88254,Operation og Intensiv (SR_ANÆS),50-60,Plejepersonale (8M_02),Sygefravær,Kvinde,67.88,10,2024-02-15,2024,02,15,0,3,Thursday,0,1
121178,Operation og Intensiv (SR_ANÆS),40-50,Plejepersonale (8M_02),Sygefravær,Kvinde,2.75,3,2024-02-15,2024,02,15,0,3,Thursday,0,1


### making a list of the day before

In [76]:
# Group by the specified columns and use shift to get the previous day's 'Antal_timer'
sygefrav.sort_values(by=['LoenafdelingNiveau2Tekst', 'Koen', 'Aldersgruppe', 'Stilling_niv1', 'Timestamp'], inplace=True) # Sort the DataFrame by the grouping columns and date to ensure correct shifting
sygefrav['Antal_timer_yesterday'] = sygefrav.groupby(['LoenafdelingNiveau2Tekst', 'Koen', 'Aldersgruppe', 'Stilling_niv1'])['Antal_Timer'].shift(1)
sygefrav.head(3)

,LoenafdelingNiveau2Tekst,Aldersgruppe,Stilling_niv1,Fravaeraarsag_gruppe,Koen,Antal_Timer,Antal_Personer,Timestamp,Year,Month,Day,is_weekend,weekday,weekday_name,is_holiday,is_school_holiday,Antal_timer_yesterday
85741,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,10.0,1,2021-01-01,2021,01,01,0,4,Friday,0,0,NaN
109604,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,26.0,2,2021-01-02,2021,01,02,1,5,Saturday,0,0,10.0
54450,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,0.0,1,2021-01-03,2021,01,03,1,6,Sunday,0,0,26.0


In [77]:
# drop NAs in ANtal_timer_yesterday
sygefrav = sygefrav.dropna(subset=['Antal_timer_yesterday'])

In [78]:
sygefrav

,LoenafdelingNiveau2Tekst,Aldersgruppe,Stilling_niv1,Fravaeraarsag_gruppe,Koen,Antal_Timer,Antal_Personer,Timestamp,Year,Month,Day,is_weekend,weekday,weekday_name,is_holiday,is_school_holiday,Antal_timer_yesterday
109604,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,26.0,2,2021-01-02,2021,01,02,1,5,Saturday,0,0,10.0
54450,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,0.0,1,2021-01-03,2021,01,03,1,6,Sunday,0,0,26.0
83308,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,10.0,1,2021-01-04,2021,01,04,0,0,Monday,0,0,0.0
69553,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,10.0,1,2021-01-10,2021,01,10,1,6,Sunday,0,0,10.0
92053,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,10.0,1,2021-01-19,2021,01,19,0,1,Tuesday,0,0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22500,Øvrige (SR_ØVR),60-70,Øvr.personale (8M_08),Sygefravær,Mand,7.4,1,2022-11-28,2022,11,28,0,0,Monday,0,0,7.4
83893,Øvrige (SR_ØVR),60-70,Øvr.personale (8M_08),Sygefravær,Mand,7.4,1,2022-11-29,2022,11,29,0,1,Tuesday,0,0,7.4
85730,Øvrige (SR_ØVR),60-70,Øvr.personale (8M_08),Sygefravær,Mand,7.4,1,2022-11-30,2022,11,30,0,2,Wednesday,0,0,7.4
67297,Øvrige (SR_ØVR),60-70,Øvr.personale (8M_08),Sygefravær,Mand,7.4,1,2022-12-01,2022,12,01,0,3,Thursday,0,0,7.4


# Merging w Arbejdstimer

In [79]:
# read in csv
arbejdstimer = pd.read_csv('/Users/laura/Desktop/data_dat_sci_exam/Arbejdstimer.csv', sep=';')
arbejdstimer = arbejdstimer.drop(columns=['LoenafdelingNiveau1Tekst']) # just displaying that it is Gødstrup


We should be aware that the data for arbejdstimer goes longer back than that of sygefravær. Want we need from this data is an estimate of how many hours the employees work grouped by lønafdeling, age and position.

In [80]:
arbejdstimer.dtypes

Dato_Regnskabsmaaned         object
Dato_Praestationsmaaned      object
LoenafdelingNiveau2Tekst     object
AldersGruppeAar              object
StillingsNiveau1             object
Koen                         object
Antal_Loentimer             float64
dtype: object

In [81]:
# calculate mean Antal_Loentimer grouped by StillingsNiveau1, AldersGruppeAar, Koen and LoenafdelingNiveau2Tekst 
dat = arbejdstimer.drop(columns=['Dato_Regnskabsmaaned', 'Dato_Praestationsmaaned'])
mean_hours = dat.groupby(['StillingsNiveau1', 'AldersGruppeAar', 'Koen', 'LoenafdelingNiveau2Tekst']).mean().reset_index()
# rename StillingsNiveau1 to Stilling_niv1 to match sygefravær
mean_hours = mean_hours.rename(columns={'StillingsNiveau1': 'Stilling_niv1',
                                        'AldersGruppeAar': 'Aldersgruppe'})

In [82]:
mean_hours

,Stilling_niv1,Aldersgruppe,Koen,LoenafdelingNiveau2Tekst,Antal_Loentimer
0,Adm.personale (8M_03),10-20,Kvinde,Flytteorganisationen (SR_FLYT),34.333333
1,Adm.personale (8M_03),10-20,Kvinde,Projekter (SR_PROJ),20.625000
2,Adm.personale (8M_03),10-20,Kvinde,Æ - Gamle/udgåede (SR_Æ),14.631429
3,Adm.personale (8M_03),10-20,Mand,Mave- og Tarmkirurgi (SR_KIRU),11.500000
4,Adm.personale (8M_03),10-20,Mand,Projekter (SR_PROJ),42.466667
...,...,...,...,...,...
1041,Øvr.sundh.pers (8M_04),70-80,Kvinde,Røntgen og Skanning (SR_RØNT),6.500000
1042,Øvr.sundh.pers (8M_04),70-80,Kvinde,Staben (SR_ADMC),3.000000
1043,Øvr.sundh.pers (8M_04),70-80,Kvinde,Timelønnede/Vagtlisten (SR_TML),50.126379
1044,Øvr.sundh.pers (8M_04),70-80,Mand,Arbejdsmedicin (SR_ARMK),32.388889


In [83]:
# display max and min value of Antal_Loentimer
print(mean_hours['Antal_Loentimer'].max())
print(mean_hours['Antal_Loentimer'].min())

4074.2263157894736
1.48


In [84]:
# append the mean Antal_Loentimer to the sygefrav dataframe
sygefrav = sygefrav.merge(mean_hours, on=['Stilling_niv1', 'Aldersgruppe', 'Koen', 'LoenafdelingNiveau2Tekst'])


In [85]:
sygefrav

,LoenafdelingNiveau2Tekst,Aldersgruppe,Stilling_niv1,Fravaeraarsag_gruppe,Koen,Antal_Timer,Antal_Personer,Timestamp,Year,Month,Day,is_weekend,weekday,weekday_name,is_holiday,is_school_holiday,Antal_timer_yesterday,Antal_Loentimer
0,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,26.0,2,2021-01-02,2021,01,02,1,5,Saturday,0,0,10.0,1173.233835
1,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,0.0,1,2021-01-03,2021,01,03,1,6,Sunday,0,0,26.0,1173.233835
2,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,10.0,1,2021-01-04,2021,01,04,0,0,Monday,0,0,0.0,1173.233835
3,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,10.0,1,2021-01-10,2021,01,10,1,6,Sunday,0,0,10.0,1173.233835
4,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,10.0,1,2021-01-19,2021,01,19,0,1,Tuesday,0,0,10.0,1173.233835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137421,Øvrige (SR_ØVR),60-70,Øvr.personale (8M_08),Sygefravær,Mand,7.4,1,2022-11-28,2022,11,28,0,0,Monday,0,0,7.4,82.918750
137422,Øvrige (SR_ØVR),60-70,Øvr.personale (8M_08),Sygefravær,Mand,7.4,1,2022-11-29,2022,11,29,0,1,Tuesday,0,0,7.4,82.918750
137423,Øvrige (SR_ØVR),60-70,Øvr.personale (8M_08),Sygefravær,Mand,7.4,1,2022-11-30,2022,11,30,0,2,Wednesday,0,0,7.4,82.918750
137424,Øvrige (SR_ØVR),60-70,Øvr.personale (8M_08),Sygefravær,Mand,7.4,1,2022-12-01,2022,12,01,0,3,Thursday,0,0,7.4,82.918750


In [86]:
# print earliest day in sygefrav
print(sygefrav['Timestamp'].max())

# print the dataframe for latest day in sygefrav
sygefrav[sygefrav['Timestamp'] == '2024-02-15'].head(6)


2024-05-09 00:00:00


,LoenafdelingNiveau2Tekst,Aldersgruppe,Stilling_niv1,Fravaeraarsag_gruppe,Koen,Antal_Timer,Antal_Personer,Timestamp,Year,Month,Day,is_weekend,weekday,weekday_name,is_holiday,is_school_holiday,Antal_timer_yesterday,Antal_Loentimer
355,Akutafdelingen (SR_FAME),20-30,Lægepersonale (8M_01),Sygefravær,Kvinde,24.00,2,2024-02-15,2024,02,15,0,3,Thursday,0,1,3.00,1173.233835
1401,Akutafdelingen (SR_FAME),20-30,Plejepersonale (8M_02),Sygefravær,Kvinde,8.25,1,2024-02-15,2024,02,15,0,3,Thursday,0,1,0.00,1329.997105
1621,Akutafdelingen (SR_FAME),30-40,Lægepersonale (8M_01),Sygefravær,Kvinde,0.00,1,2024-02-15,2024,02,15,0,3,Thursday,0,1,25.50,519.952320
2811,Akutafdelingen (SR_FAME),30-40,Plejepersonale (8M_02),Sygefravær,Kvinde,8.00,5,2024-02-15,2024,02,15,0,3,Thursday,0,1,8.25,1456.170923
4288,Akutafdelingen (SR_FAME),40-50,Plejepersonale (8M_02),Sygefravær,Kvinde,10.00,2,2024-02-15,2024,02,15,0,3,Thursday,0,1,8.50,1634.982500
6259,Akutafdelingen (SR_FAME),50-60,Plejepersonale (8M_02),Sygefravær,Kvinde,0.00,1,2024-02-15,2024,02,15,0,3,Thursday,0,1,0.00,1467.242540


In [87]:
# write sygefrav to csv
sygefrav = sygefrav.drop('weekday_name', axis=1)
sygefrav.to_csv('/Users/laura/Desktop/data_dat_sci_exam/fravær_merge.csv', sep=';', index=False)